# 1. Extração dos Enunciados das Provas do Enem LEDOR (2015-2023)

Este notebook apresenta o processo de extração textual das provas do ENEM adaptadas para acessibilidade, conhecidas como provas LEDOR. Essas versões são destinadas a participantes com deficiência visual e trazem descrições detalhadas de imagens e gráficos, além de uma estrutura mais textual e padronizada.

Optamos por utilizar as provas LEDOR ao invés da aplicação regular devido aos seguintes benefícios para a nossa tarefas de PLN e predição:

- Enunciados mais completos e com menos dependência de elementos visuais;
- Formato mais limpo, facilitando a tokenização, análise semântica e a modelagem textual;
- Melhor desempenho na extração automática com bibliotecas como PyMuPDF, em conjunto com regex e revisão manual.


In [16]:
# Importando as bibliotecas necessárias para extração de texto
import re
import pandas as pd
from PyPDF2 import PdfReader

## 1.1. Funções Auxiliares

In [17]:
# Função que recebe o caminho do pdf, a página de início e a de fim e retorna as questões
def extract_questions(full_text: str, caps_lock: bool = False) -> list[str]:
    delimiter = "QUESTÃO " if caps_lock else "Questão "
    sections = full_text.split(delimiter)
    questions = [s[3:] for s in sections if s[:2].isnumeric()]
    return questions

# Extração dos enunciados
def get_questions_text(pdf_path: str, start_page: int, end_page: int, caps_lock: bool = False) -> list[str]:
    reader = PdfReader(pdf_path)
    final_questions = []

    for page_num in range(start_page, end_page):
        full_text = reader.pages[page_num].extract_text() or ""
        final_questions.extend(extract_questions(full_text, caps_lock))

    return final_questions

In [18]:
# Função que retorna as questões sem as alternativas e as alternativas formatadas
def format_alternatives(alt: str) -> str:
    match = re.search(r"(A\s.+?)(B\s.+?)(C\s.+?)(D\s.+?)(E\s.+)", alt)
    if match:
        groups = match.groups()
        return "; ".join([f"{item[0]}: {item[2:-1]}" for item in groups])
    return ""


def get_alternatives(questions: list[str], filter: bool = False) -> tuple[list[str], list[str]]:
    formatted_questions = []
    formatted_alternatives = []

    for question in questions:
        parts = re.split(r"(\nA\s.*\n)", question)

        alternatives_text = "".join(parts[-2:]).replace("\n", "")
        alternatives_text = re.sub(r"\*.*\*", "", alternatives_text)
        alternatives_text = re.sub(r"(CH).*\d", "", alternatives_text)
        alternatives = format_alternatives(alternatives_text)

        question_text = "".join(parts[:-2]).replace("\n", "")

        formatted_questions.append(question_text)
        formatted_alternatives.append(alternatives)

    return formatted_questions, formatted_alternatives

In [19]:
# Função que realiza a leitura dos microdados
def reading_data(path_to_data: str, year: str, code: int) -> pd.DataFrame:

    df = pd.read_csv(path_to_data, sep=';', encoding='latin-1')
    
    df = df[(df['SG_AREA'] == 'CH') & (df['TX_COR'].str.lower() == 'laranja')]
    df = df[df['CO_PROVA'] == code]
    df = df[['CO_POSICAO', 'TX_GABARITO', 'NU_PARAM_A', 'NU_PARAM_B', 'NU_PARAM_C']].copy()
    
    df['ANO'] = year

    return df

# Função auxiliar para criar o dataset com as novas colunas

def merge_questions_alternatives(dataset: pd.DataFrame, questions: list[str], alts: list[str]) -> pd.DataFrame:
    dataset['QUESTOES'] = questions
    dataset['ALTERNATIVAS'] = alts

    return dataset

## 1.2. Extração do Texto e Cruzamento com Microdados

In [20]:
years = {
    2017: (19, 32, True, 408),
    2018: (19, 32, True, 464),
    2019: (19, 32, False, 520),
    2020: (19, 32, False, 574),
    2021: (23, 36, False, 886),
    2022: (20, 32, True, 1062),
    2023: (19, 32, True, 1198),
}

In [21]:
for year, (start_page, end_page, question_caps, code) in years.items():
    path_to_microdados = f"../data/raw/microdados/microdados_{year}.csv"
    path_to_prova = f"../data/raw/provas/Enem_{year}.pdf"
    output_path = f"../data/processed/enem_{year}.csv"

    questions = get_questions_text(path_to_prova, start_page, end_page, question_caps)
    questions, alternatives = get_alternatives(questions)

    dataset = reading_data(path_to_microdados, year, code)

    if year in (2022, 2023):
        questions = [question for question in questions if question]
        alternatives = [alternative for alternative in alternatives if alternative]
        
    dataset = merge_questions_alternatives(dataset, questions, alternatives)

    dataset = dataset.rename(
        columns={
            "CO_POSICAO": "numero_questao",
            "QUESTOES": "enunciado",
            "ALTERNATIVAS": "alternativas",
            "NU_PARAM_B": "nu_param_B",
            "TX_GABARITO": "gabarito",
        }
    )

    dataset.to_csv(output_path, sep=";", index=False)
    print(f"Arquivo {output_path} criado com sucesso.")

Arquivo ../data/processed/enem_2017.csv criado com sucesso.
Arquivo ../data/processed/enem_2018.csv criado com sucesso.
Arquivo ../data/processed/enem_2019.csv criado com sucesso.
Arquivo ../data/processed/enem_2020.csv criado com sucesso.
Arquivo ../data/processed/enem_2021.csv criado com sucesso.
Arquivo ../data/processed/enem_2022.csv criado com sucesso.
Arquivo ../data/processed/enem_2023.csv criado com sucesso.


## 1.3. Unindo Todos os Dados em Único Arquivo

In [22]:
dataframes = {}
output_path = "../data/final/enem_data.csv"

for year in years:
    df_path = f"../data/processed/enem_{year}.csv"
    df = pd.read_csv(df_path, encoding="utf-8", quotechar='"', sep=';', index_col=0)
    dataframes[year] = df

In [23]:
df_final = pd.concat(dataframes.values(), axis=0)

df_final.to_csv(output_path)

In [24]:
df_final = pd.read_csv(output_path)
df_final.head()

,numero_questao,gabarito,NU_PARAM_A,nu_param_B,NU_PARAM_C,ANO,enunciado,alternativas
0,1,C,3.43894,0.97831,0.10855,2017,"No império africano do Mali, no século XIV, To...",A: isolamento geográﬁco do Saara ocidental; B...
1,2,D,3.00837,0.49169,0.13877,2017,Após a Declaração Universal dos Direitos Human...,A: ataque feito pelos japoneses à base milita...
2,3,D,0.60432,3.25992,0.08798,2017,"A moralidade, Bentham exortava, não é uma ques...",A: fundamentação cientíﬁca de viés positivist...
3,4,E,1.85031,0.57925,0.11344,2017,Fala-se muito nos dias de hoje em direitos do ...,A: modernização da educação escolar; B: atuali...
4,5,C,2.46290,0.76307,0.17672,2017,Na Constituição da República Federativa do Bra...,A: etnia e miscigenação racial; B: sociedade...
